In [122]:
%load_ext jupyter_black

import pandas as pd
import os
from pathlib import Path

# input
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/03_rainfall/input"
)

# outputs
processed_output_dir = os.path.join(
    input_dir, "gpm_data/rainfall_data/output_hhr_processed/"
)

output_dir = os.path.join(
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/03_rainfall/output"
)

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [123]:
# loading csv files
typ_list = [
    x for x in os.listdir(processed_output_dir) if x.endswith("_gridstats.csv")
]
# typ_file = pd.read_csv(processed_output_dir + typ_list[0])
# typ_file

In [124]:
for typ in typ_list:
    typ_file = pd.read_csv(processed_output_dir + typ)
    sorted_df = typ_file.sort_values(by=["id", "date", "end"])
    # computing 6hr and 24hr max in mm/hr
    six_hr_max_df = (
        sorted_df.groupby(["id", "Centroid"])[["mean", "max"]]
        .rolling(2)
        .sum()
        .rolling(6)
        .max()
        .reset_index()
        .drop("level_2", axis=1)
        .rename(
            {"max": "six_hr_max_of_max", "mean": "six_hr_max_of_mean"}, axis=1
        )
    )
    day_max_df = (
        sorted_df.groupby(["id", "Centroid"])[["mean", "max"]]
        .rolling(2)
        .sum()
        .rolling(24)
        .max()
        .reset_index()
        .drop("level_2", axis=1)
        .rename({"max": "day_max_of_max", "mean": "day_max_of_mean"}, axis=1)
    )
    # joining all output
    out_df = six_hr_max_df.reset_index(drop=True).join(
        day_max_df.drop(["id", "Centroid"], axis=1).join(
            sorted_df[["end", "date"]].reset_index().drop("index", axis=1)
        )
    )
    out_df["typhoon_name"] = typ.split("_")[0]
    out_df.to_csv(
        output_dir + "/" + typ.split("_")[0] + "_stats.csv", index=False
    )